### IBM Data Science Capstone Project
#### The Battle of Neighborhoods

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [98]:
URL = "https://www.zip-codes.com/zip-code-radius-finder.asp?zipmileslow=0&zipmileshigh=30&zip1=03869&submit=Search"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find("div", {"id" : "tableview"})
list_table = tableDataText(table)
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df = df.drop('#', axis = 1)
df = df[:-1]
df.tail()

,Zip,City,County,St,Country,Distance,
98,04061,NORTH WATERBORO,YORK,ME,US,29.18 miles,
99,03810,ALTON BAY,BELKNAP,NH,US,29.38 miles,
100,04006,BIDDEFORD POOL,YORK,ME,US,29.53 miles,
101,04056,NEWFIELD,YORK,ME,US,29.56 miles,
102,04063,OCEAN PARK,YORK,ME,US,29.60 miles,


In [99]:
def getIncomeData(base_URL, num_pages):
    df_income = pd.DataFrame()
    i = 0
    while i<num_pages:
        if i == 0:
            URL = base_URL.format('')
        else:
            URL = base_URL.format(str(i+1) + ".")
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find('div', {'id' : 'ctl00_ContentPlaceHolder1_final_content'})
        income_table = tableDataText(table)
        df_income = df_income.append(pd.DataFrame(income_table[4:], columns=income_table[3]))
        i += 1
    return df_income[:-3]

In [100]:
me_income = getIncomeData("http://zipatlas.com/us/me/zip-code-comparison/median-household-income.{}htm", 4)
nh_income = getIncomeData("http://zipatlas.com/us/nh/zip-code-comparison/median-household-income.{}htm", 3)
income_data = nh_income.append(me_income)
income_data[['Latitude', 'Longitude']] = income_data['Location'].str.split(',', expand = True)
income_data = income_data.drop(['#', 'Location', 'National Rank'], axis = 1)
income_data = income_data.rename(columns = {'Avg. Income/H/hold': 'Household Income'})
income_data.head()

,Zip Code,City,Population,Household Income,Latitude,Longitude
0,03087,"Windham,New Hampshire","10,709","$94,794.00",42.811092,-71.302688
1,03049,"Hollis,New Hampshire","7,017","$92,847.00",42.749626,-71.585401
2,03031,"Amherst,New Hampshire","10,812","$88,671.00",42.874854,-71.600512
3,03750,"Etna,New Hampshire",962,"$86,421.00",43.708188,-72.195047
4,03233,"Elkins,New Hampshire",234,"$85,247.00",43.438900,-71.952889


In [101]:
df = pd.merge(df, income_data,how="inner", left_on="Zip", right_on = "Zip Code")
df = df.drop(['City_x', 'County', 'St', 'Country', 'Zip'], axis = 1)
df = df.rename(columns = {'City_y': 'City'})
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])
df.head()

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112
1,3.20 miles,,03878,"Somersworth,New Hampshire","11,483","$43,177.00",43.253476,-70.887944
2,3.52 miles,,03820,"Dover,New Hampshire","28,537","$44,493.00",43.187186,-70.902839
3,5.29 miles,,03908,"South Berwick,Maine","6,682","$53,294.00",43.230492,-70.752426
4,5.83 miles,,03901,"Berwick,Maine","6,338","$44,519.00",43.293960,-70.836455


In [106]:
rollinsford = df[df['City'] == 'Rollinsford,New Hampshire']
map_rollinsford = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rollinsford)  

map_rollinsford

In [119]:
CLIENT_ID = 'W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ' # your Foursquare ID
CLIENT_SECRET = 'T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ' # your Foursquare Secret
ACCESS_TOKEN = 'TTU5B5RAIMXLGMKTTL0YW2K0QBJJLKYYP2MMP0JQ3QCCGV3Y' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ
CLIENT_SECRET:T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ


In [121]:
radius = 3000
LIMIT = 500

In [122]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                             'City Latitude', 
                             'City Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [123]:
nearby_venues = getNearbyVenues(names = df['City'],
                                latitudes = df['Latitude'],
                                longitudes = df['Longitude'],
                                radius = 3000)
nearby_venues.shape

(1466, 7)

In [138]:
venue_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix = "", prefix_sep="")
venue_onehot['City'] = nearby_venues['City']

tmp = venue_onehot.pop('City')
venue_onehot.insert(0, 'City', tmp)

city_venues = venue_onehot.groupby('City').mean().reset_index()
city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Acton,Maine",0.0,0.0,0.2,0.200000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alfred,Maine",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alton Bay,New Hampshire",0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Alton,New Hampshire",0.0,0.0,0.0,0.210526,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Barnstead,New Hampshire",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
print(list(city_venues.columns))

['City', 'Accessories Store', 'Airport', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Arcade', 'Art Gallery', 'Arts & Crafts Store', 'Asian Restaurant', 'Assisted Living', 'Athletics & Sports', 'Automotive Shop', 'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field', 'Basketball Court', 'Beach', 'Big Box Store', 'Bistro', 'Board Shop', 'Boat Launch', 'Boat or Ferry', 'Bookstore', 'Border Crossing', 'Bowling Alley', 'Breakfast Spot', 'Brewery', 'Bridge', 'Burger Joint', 'Bus Station', 'Business Service', 'Butcher', 'Cafeteria', 'Café', 'Campground', 'Candy Store', 'Caribbean Restaurant', 'Chinese Restaurant', 'Chocolate Shop', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Cafeteria', 'College Library', 'Comic Shop', 'Construction & Landscaping', 'Convenience Store', 'Cosmetics Shop', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Diner', 'Disc Golf', 'Discount Store', 'Dog Run', 'Donut Shop', 'Drive-in Theater', 'Electroni

In [134]:
good_cols = ['Athletics & Sports', 'Dance Studio','Golf Course','Gymnastics Gym','Health Food Store','Juice Bar','Sporting Goods Shop','Soccer Field','Trail','Weight Loss Center','Yoga Studio']
bad_cols = ['Gym','Gym / Fitness Center']

In [148]:
city_venues['Good columns'] = city_venues[good_cols].sum(axis = 1)
city_venues['Bad columns'] = city_venues[bad_cols].sum(axis = 1)
sorted_city_venues = city_venues.sort_values(by='Good columns', ascending = False)
sorted_city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Good columns,Bad columns
67,"Suncook,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0
47,"Nottingham,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.0
56,"Saco,Maine",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.0
40,"Newfields,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0
37,"Milton Mills,New Hampshire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0


In [152]:
df_good_bad_cols = sorted_city_venues[['City','Good columns', 'Bad columns']]
df_info = pd.merge(df, df_good_bad_cols,how="inner", on = 'City')
df_info.head()

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude,Good columns,Bad columns
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112,0.136364,0.000000
1,3.20 miles,,03878,"Somersworth,New Hampshire","11,483","$43,177.00",43.253476,-70.887944,0.053571,0.017857
2,3.52 miles,,03820,"Dover,New Hampshire","28,537","$44,493.00",43.187186,-70.902839,0.015385,0.015385
3,5.29 miles,,03908,"South Berwick,Maine","6,682","$53,294.00",43.230492,-70.752426,0.000000,0.500000
4,5.83 miles,,03901,"Berwick,Maine","6,338","$44,519.00",43.293960,-70.836455,0.000000,0.000000


In [153]:
df_info.sort_values('Good columns', ascending = False)

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude,Good columns,Bad columns
72,28.96 miles,,03275,"Suncook,New Hampshire","11,764","$46,513.00",43.166029,-71.416310,0.500000,0.000000
29,15.48 miles,,03290,"Nottingham,New Hampshire","3,585","$63,438.00",43.129847,-71.131859,0.400000,0.000000
71,28.89 miles,,04072,"Saco,Maine","19,177","$45,857.00",43.550411,-70.466341,0.400000,0.000000
25,14.09 miles,,03856,"Newfields,New Hampshire","1,551","$71,375.00",43.038509,-70.967954,0.333333,0.000000
42,21.11 miles,,03852,"Milton Mills,New Hampshire",534,"$48,125.00",43.504782,-70.979586,0.333333,0.000000
31,15.89 miles,,03884,"Strafford,New Hampshire","1,951","$62,400.00",43.275599,-71.168991,0.250000,0.000000
46,21.56 miles,,03044,"Fremont,New Hampshire","3,522","$62,237.00",42.999857,-71.119714,0.250000,0.000000
24,13.74 miles,,03815,"Center Strafford,New Hampshire",473,"$57,614.00",43.266849,-71.107519,0.200000,0.000000
21,12.96 miles,,03840,"Greenland,New Hampshire","3,208","$62,172.00",43.027974,-70.844787,0.200000,0.000000
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112,0.136364,0.000000
